In [11]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.geocoders import ArcGIS
import re
import folium
from tqdm import tqdm
import time  # Para controlar requisições
import ipywidgets

tqdm.pandas(dynamic_ncols=True)

In [12]:
# Importação do arquivo principal
df = pd.read_csv("database/dadosTratados/df_final_semGeo.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893 entries, 0 to 892
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NOME            893 non-null    object 
 1   TIPO            893 non-null    object 
 2   ENDERECO        893 non-null    object 
 3   LOCAL           893 non-null    object 
 4   PONTUACAO       893 non-null    float64
 5   NUM_COMENTARIO  893 non-null    int64  
 6   TEMPO_MIN_M     893 non-null    float64
 7   TEMPO_MAX_M     893 non-null    float64
 8   TEMPO_MIN       893 non-null    object 
 9   TEMPO_MAX       893 non-null    object 
 10  BAIRRO          893 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 76.9+ KB


In [ ]:

# Obtendo as coordenada

# Utilizando o geocodificador da ArcGIS
geolocator = ArcGIS(timeout=10)

def obter_coordenadas(endereco):
    try:
        location = geolocator.geocode(endereco)
        time.sleep(1)  # respeita limite de 1 requisição por segundo
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except:
        return pd.Series([None, None])

# Aplica a função com barra de progresso
df[['LATITUDE', 'LONGITUDE']] = df['ENDERECO'].progress_apply(obter_coordenadas)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 893/893 [16:59<00:00,  1.14s/it]


In [ ]:
# Salva o novo dataFrame com a geolocalização
df.to_csv("database/dadosTratados/df_final_comGeo.csv")

df

,NOME,TIPO,ENDERECO,LOCAL,PONTUACAO,NUM_COMENTARIO,TEMPO_MIN_M,TEMPO_MAX_M,TEMPO_MIN,TEMPO_MAX,BAIRRO,LAT,LNG
0,Shopping RioMar Fortaleza,cafeteria,"R. Des. Lauro Nogueira, 1500 - Papicu, Fortale...","Shopping center em Fortaleza, Ceará",4.7,52356,45.0,180.0,0 days 00:45:00,0 days 03:00:00,Papicu,-3.742329,-38.471450
1,Greta Café,cafeteria,"Av. Antônio Sales, 2960 - Dionísio Torres, For...",$$,4.7,1094,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Dionísio Torres,-3.747024,-38.497650
2,Bardo Café,cafeteria,"R. Amélia Benebien, 471 - Papicu, Fortaleza - ...",Cafeteria,4.9,22,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Papicu,-3.737813,-38.476876
3,"Go Coffee: Cafés, Chás, Frappes, Chocolate Que...",cafeteria,"R. Pereira Valente, 578 - Meireles, Fortaleza ...",Cafeteria,4.6,62,10.0,10.0,0 days 00:10:00,0 days 00:10:00,Meireles,-3.732479,-38.497050
4,Le Pain Le Café | Aldeota,cafeteria,"Rua Professor Dias da Rocha, 670 - Meireles, F...",$$$,4.4,1277,20.0,90.0,0 days 00:20:00,0 days 01:30:00,Meireles,-3.735362,-38.491679
...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,Padaria Romana (Centro),restaurante,"Av. Duque de Caxias, 137 - Centro, Fortaleza -...",$$,4.2,865,15.0,15.0,0 days 00:15:00,0 days 00:15:00,Centro,-3.733001,-38.525689
889,Bolo,restaurante,"Santa rosa - Santa Rosa, Fortaleza - CE, 60703...",NaN,0.0,0,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Santa Rosa,-3.819936,-38.595282
890,Restaurante,restaurante,"R. Gama, 170 - Vila Velha, Fortaleza - CE, 603...",Restaurante,3.0,1,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Vila Velha,-3.712739,-38.600711
891,Restaurante,restaurante,"Rua Ministro Antônio Coelho - Vila Velha, Fort...",NaN,0.0,0,0.0,0.0,0 days 00:00:00,0 days 00:00:00,Vila Velha,-3.714419,-38.592349
